# Calling C++

Pyton contains multiple ways of calling functions written in C++. This notebooks shows how to use **ctypes** and **cffi** on a **Windows** computer. 

* **ctypes**: Recommended for calling C++ function *outside* **Numba**.
* **cffi**: Required to call C++ function *inside* **Numba**. Structs not allowed.


From the **consav** package we will use the **cpptools** module to compile and link to C++ files.

**Compilers:** Two compiler workflows have been implemented:

* **vs**: Free *Microsoft Visual Studio 2017 Community Edition* ([link](https://visualstudio.microsoft.com/downloads/))
* **intel:** Costly *Intel Parallel Studio 2018 Composer Edition* ([link](https://software.intel.com/en-us/parallel-studio-xe))

For parallization we will use **OpenMP**.

The **installation paths** might need to be adjusted. See arguments to the **cpptools.compile()** function.

In [1]:
compiler = 'vs'

# ctypes

In [2]:
# use 8 threads in numba
from consav import runtools
runtools.write_numba_config(disable=0,threads=8)

In [3]:
import ctypes as ct
import numpy as np
import numba as nb
from consav import cpptools

In [4]:
# a. main class

# list of elements
parlist = [
    ('X',nb.double[:]),
    ('Y',nb.double[:]),    
    ('N',nb.int32),    
    ('a',nb.double),
    ('b',nb.double),
    ('threads',nb.int32)
]
# python class
class ParClass():
    def __init__(self):
        pass

# cpp struct 

#  return python version of the C++ struct
#  write file with struct definition to include in .cpp-file (note: ensures order of fields is the same)
ParStruct = cpptools.setup_struct(parlist,structname='par_struct',structfile='cppfuncs/par_struct.cpp')

# b. compile
cpptools.compile('cppfuncs/example',compiler=compiler) # adjust paths?

# c. settings
par = ParClass()
par.N = 10
par.X = np.linspace(0,10,par.N)
par.Y = np.zeros(par.N)
par.a = 2
par.b = 1
par.threads = 4

# d. link

# list of functions with argument types (long is int)
funcs = [('fun',[ct.POINTER(ParStruct)]),
         ('fun_nostruct',[ct.POINTER(ct.c_double),
                          ct.POINTER(ct.c_double),
                          ct.c_long,
                          ct.c_double,
                          ct.c_double,
                          ct.c_long])]

if compiler == 'vs':
    example = cpptools.link('example',funcs,use_openmp_with_vs=True)
else:
    example = cpptools.link('example',funcs)

# e. wrapper
def wrapper(par):
    p_par = cpptools.get_struct_pointer(par,ParStruct)
    example.fun(p_par)

def wrapper_nostruct(X,Y,N,a,b,threads):
    p_X = np.ctypeslib.as_ctypes(X)
    p_Y = np.ctypeslib.as_ctypes(Y)
    example.fun_nostruct(p_X,p_Y,N,a,b,threads)
    
# f. calls and checks
wrapper(par)
assert np.allclose(par.X*(par.a+par.b),par.Y)
print('all assertions true for example.fun')

par.Y = np.zeros(par.N)
wrapper_nostruct(par.X,par.Y,par.N,par.a,par.b,par.threads)
assert np.allclose(par.X*(par.a+par.b),par.Y)
print('all assertions true for example.fun_nostruct')

# g. delink (remove dll file)
cpptools.delink(example,'example')

cpp files compiled
cpp files loaded
all assertions true for example.fun
all assertions true for example.fun_nostruct
cpp files delinked


# cffi

In [5]:
import os
from cffi import FFI
import numba as nb

In [6]:
# a. main class

# list of elements 
parlist = [
    ('X',nb.double[:]),
    ('Y',nb.double[:]),    
    ('N',nb.int32),    
    ('a',nb.double),
    ('b',nb.double),
    ('threads',nb.int32)
]

# python class
@nb.jitclass(parlist)
class ParClass():
    def __init__(self):
        pass

# b. compile
cpptools.compile('cppfuncs/example',compiler=compiler)

# c. settings
par = ParClass()
par.N = 10
par.X = np.zeros(par.N)
par.Y = np.zeros(par.N)
par.a = 2
par.b = 1
par.threads = 4

# d. link
ffi = FFI()
ffi.cdef(r'''void fun_nostruct(double *X, double *Y, int N, double a, double b, int threads);''')
example = ffi.dlopen("example.dll")

# e. regular call
p_X = ffi.cast('double *', par.X.ctypes.data)
p_Y = ffi.cast('double *', par.Y.ctypes.data)
example.fun_nostruct(p_X,p_Y,par.N,par.a,par.b,par.threads)
assert np.allclose(par.X*(par.a+par.b),par.Y)
print('all assertions true for fun_nostruct')

# f. numba call
fun_nostruct_numba = example.fun_nostruct
@nb.njit
def wrapper_nostruct(X,Y,N,a,b,threads):
    p_X = ffi.from_buffer(X)
    p_Y = ffi.from_buffer(Y)
    fun_nostruct_numba(p_X,p_Y,N,a,b,threads)

par.Y = np.zeros(par.N)
wrapper_nostruct(par.X,par.Y,par.N,par.a,par.b,par.threads)
assert np.allclose(par.X*(par.a+par.b),par.Y)
print('all assertions true for fun_nostruct (in numba)')

# g. clean up
ffi.dlclose(example)
os.remove('example.dll')

cpp files compiled
all assertions true for fun_nostruct
all assertions true for fun_nostruct (in numba)
